# Feature engineering

In [1]:
%config Completer.use_jedi = False
import pandas as pd
import torch
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import numpy as np
from torch import nn
from torch import optim
from torch.nn import functional as F
from torch.optim.lr_scheduler import _LRScheduler
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt
from multiprocessing import cpu_count
from pathlib import Path
import tsfresh
from tsfresh.feature_extraction import EfficientFCParameters, MinimalFCParameters, ComprehensiveFCParameters
from tsfresh.utilities.dataframe_functions import roll_time_series

pd.set_option('display.max_rows', 10000)

In [2]:
data = pd.read_csv('./FIT5149_A2_data/train_data_withlabels.csv')
data.rename(columns={"Unnamed: 0":"id"}, inplace= True)


In [3]:
data.head(1000)

,id,load,ac,ev,oven,wash,dryer,hourofday,dayofweek,dif,absdif,max,var,entropy,nonlinear,hurst
0,105541,2.245,0,0,0,0,0,0,Sun,9.870000e-01,9.870000e-01,6.215,3.074549,0.678886,0.052903,0.994071
1,105542,2.259,0,0,0,0,0,0,Sun,1.400000e-02,1.400000e-02,6.215,3.172867,0.667450,0.054829,0.994154
2,105543,2.269,0,0,0,0,0,0,Sun,1.000000e-02,1.000000e-02,6.215,3.270112,0.647777,0.056991,0.994220
3,105544,2.268,0,0,0,0,0,0,Sun,-1.000000e-03,1.000000e-03,6.215,3.303763,0.629227,0.057606,0.994150
4,105545,2.270,0,0,0,0,0,0,Sun,2.000000e-03,2.000000e-03,6.215,3.302744,0.621295,0.082640,0.994041
5,105546,2.259,0,0,0,0,0,0,Sun,-1.100000e-02,1.100000e-02,6.215,3.298258,0.626956,0.089788,0.993986
6,105547,2.250,0,0,0,0,0,0,Sun,-9.000000e-03,9.000000e-03,6.215,3.289135,0.640121,0.085276,0.993927
7,105548,2.251,0,0,0,0,0,0,Sun,1.000000e-03,1.000000e-03,6.215,3.238982,0.651916,0.354203,0.992703
8,105549,2.264,0,0,0,0,0,0,Sun,1.300000e-02,1.300000e-02,6.215,3.203236,0.655600,0.372529,0.992474
9,105550,2.277,0,0,0,0,0,0,Sun,1.300000e-02,1.300000e-02,6.215,3.196127,0.650006,0.366578,0.992308


In [4]:
data.dtypes

id             int64
load         float64
ac             int64
ev             int64
oven           int64
wash           int64
dryer          int64
hourofday      int64
dayofweek     object
dif          float64
absdif       float64
max          float64
var          float64
entropy      float64
nonlinear    float64
hurst        float64
dtype: object

In [5]:
x_train = data.loc[:,data.columns.difference(['ac', 'ev', 'oven', 'wash', 'dryer'])]
# x_train.drop('Unnamed: 0', axis = 1, inplace = True)
y_train = data[['ac', 'ev', 'oven', 'wash', 'dryer']]

In [6]:
le = preprocessing.LabelEncoder()
le.fit(x_train['dayofweek'])
le.classes_
x_train['dayofweek'] = le.transform(x_train['dayofweek'])

In [7]:
x_train['dummy_id'] = 1
x_train.columns

Index(['absdif', 'dayofweek', 'dif', 'entropy', 'hourofday', 'hurst', 'id',
       'load', 'max', 'nonlinear', 'var', 'dummy_id'],
      dtype='object')

In [8]:
# Scale
scale_list = ['absdif', 'dayofweek', 'dif', 'entropy', 'hourofday', 'hurst',
       'load', 'max', 'nonlinear', 'var']
scaler = preprocessing.StandardScaler().fit(x_train[scale_list])
x_train[scale_list] = scaler.transform(x_train[scale_list])
x_train

,absdif,dayofweek,dif,entropy,hourofday,hurst,id,load,max,nonlinear,var,dummy_id
0,1.633999,-0.006818,1.859011,-0.306037,-1.659524,0.325913,105541,0.031914,1.050126,-0.542338,0.673127,1
1,-0.287489,-0.006818,0.026370,-0.427219,-1.659524,0.327174,105542,0.039319,1.050126,-0.541600,0.728126,1
2,-0.295388,-0.006818,0.018836,-0.635696,-1.659524,0.328183,105543,0.044609,1.050126,-0.540772,0.782525,1
3,-0.313161,-0.006818,-0.001882,-0.832269,-1.659524,0.327108,105544,0.044080,1.050126,-0.540536,0.801349,1
4,-0.311186,-0.006818,0.003768,-0.916324,-1.659524,0.325441,105545,0.045138,1.050126,-0.530947,0.800779,1
...,...,...,...,...,...,...,...,...,...,...,...,...
417715,-0.309212,0.992434,-0.005649,-7.500160,1.375005,-14.864864,523256,0.189539,-1.866220,-0.562601,-1.046775,1
417716,-0.066310,0.992434,-0.237319,-7.500160,1.375005,-14.864864,523257,0.122892,-1.866220,-0.562601,-1.046775,1
417717,2.485141,0.992434,-2.670795,-7.500160,1.375005,-14.864864,523258,-0.627149,-1.866220,-0.562601,-1.046775,1
417718,-0.249967,0.992434,-0.062154,-7.500160,1.375005,-14.864864,523259,-0.644604,-1.866220,-0.562601,-1.046775,1


In [9]:
extra_len = pd.DataFrame(x_train.iloc[0:1,:].values.repeat(29, axis = 0), columns=x_train.columns)
extra_len['id'] = list(range(105541-29,105541))
x_train = pd.concat([extra_len, x_train])
x_train

,absdif,dayofweek,dif,entropy,hourofday,hurst,id,load,max,nonlinear,var,dummy_id
0,1.633999,-0.006818,1.859011,-0.306037,-1.659524,0.325913,105512,0.031914,1.050126,-0.542338,0.673127,1.0
1,1.633999,-0.006818,1.859011,-0.306037,-1.659524,0.325913,105513,0.031914,1.050126,-0.542338,0.673127,1.0
2,1.633999,-0.006818,1.859011,-0.306037,-1.659524,0.325913,105514,0.031914,1.050126,-0.542338,0.673127,1.0
3,1.633999,-0.006818,1.859011,-0.306037,-1.659524,0.325913,105515,0.031914,1.050126,-0.542338,0.673127,1.0
4,1.633999,-0.006818,1.859011,-0.306037,-1.659524,0.325913,105516,0.031914,1.050126,-0.542338,0.673127,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
417715,-0.309212,0.992434,-0.005649,-7.500160,1.375005,-14.864864,523256,0.189539,-1.866220,-0.562601,-1.046775,1.0
417716,-0.066310,0.992434,-0.237319,-7.500160,1.375005,-14.864864,523257,0.122892,-1.866220,-0.562601,-1.046775,1.0
417717,2.485141,0.992434,-2.670795,-7.500160,1.375005,-14.864864,523258,-0.627149,-1.866220,-0.562601,-1.046775,1.0
417718,-0.249967,0.992434,-0.062154,-7.500160,1.375005,-14.864864,523259,-0.644604,-1.866220,-0.562601,-1.046775,1.0


In [10]:
# x_train_rolled = roll_time_series(x_train, column_id="dummy_id", column_sort="id",
#                             max_timeshift = 29, min_timeshift = 29)
# np.save("np_x_train_rolled_new.npy",np.array(x_train_rolled))
x_train_rolled = np.load('./np_x_train_rolled_new.npy', allow_pickle= True)
x_train_rolled = pd.DataFrame(x_train_rolled, columns=x_train.columns)
x_train_rolled = x_train_rolled[['id', 'load', 'absdif', 'dayofweek', 'dif', 'entropy', 'hourofday', 'hurst', 
        'max', 'nonlinear', 'var']]
x_train_rolled

,id,load,absdif,dayofweek,dif,entropy,hourofday,hurst,max,nonlinear,var
0,"(1, 105541)",0.031914,1.633999,-0.006818,1.859011,-0.306037,-1.659524,0.325913,1.050126,-0.542338,0.673127
1,"(1, 105541)",0.031914,1.633999,-0.006818,1.859011,-0.306037,-1.659524,0.325913,1.050126,-0.542338,0.673127
2,"(1, 105541)",0.031914,1.633999,-0.006818,1.859011,-0.306037,-1.659524,0.325913,1.050126,-0.542338,0.673127
3,"(1, 105541)",0.031914,1.633999,-0.006818,1.859011,-0.306037,-1.659524,0.325913,1.050126,-0.542338,0.673127
4,"(1, 105541)",0.031914,1.633999,-0.006818,1.859011,-0.306037,-1.659524,0.325913,1.050126,-0.542338,0.673127
...,...,...,...,...,...,...,...,...,...,...,...
12531595,"(1, 523260)",0.189539,-0.309212,0.992434,-0.005649,-7.50016,1.375005,-14.864864,-1.86622,-0.562601,-1.046775
12531596,"(1, 523260)",0.122892,-0.06631,0.992434,-0.237319,-7.50016,1.375005,-14.864864,-1.86622,-0.562601,-1.046775
12531597,"(1, 523260)",-0.627149,2.485141,0.992434,-2.670795,-7.50016,1.375005,-14.864864,-1.86622,-0.562601,-1.046775
12531598,"(1, 523260)",-0.644604,-0.249967,0.992434,-0.062154,-7.50016,1.375005,-14.864864,-1.86622,-0.562601,-1.046775


In [11]:
print(len(x_train_rolled)/30)
print(len(x_train)-29)

417720.0
417720


In [12]:
y_train['transformed'] = y_train.apply(lambda x: ''.join(x.astype(str)),axis = 1)
le_y = preprocessing.LabelEncoder()
y_train['encoded'] = le_y.fit_transform(y_train['transformed'])
y_train_transformed = y_train['encoded']
y_train_transformed = np.array(y_train_transformed)
y_train_transformed

<ipython-input-12-0b76bc136e4a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_train['transformed'] = y_train.apply(lambda x: ''.join(x.astype(str)),axis = 1)
<ipython-input-12-0b76bc136e4a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_train['encoded'] = le_y.fit_transform(y_train['transformed'])


0         0
1         0
2         0
3         0
4         0
         ..
417715    0
417716    0
417717    0
417718    0
417719    0
Name: encoded, Length: 417720, dtype: int64

In [13]:
x_train_rolled.drop('id', axis=1, inplace=True)
x_train_rolled = np.reshape(x_train_rolled.to_numpy(dtype=np.float64), [-1 , 30, int(x_train_rolled.shape[1])])
x_train_rolled.shape #(417720, 30, 10)

(417720, 30, 10)

In [14]:
def absfft(x):
    return np.abs(np.fft.rfft(x))
x_train_fft = np.copy(x_train_rolled)
x_train_fft = np.apply_along_axis(absfft, 1, x_train_fft)
x_train_fft

array([[[9.57422367e-01, 4.90199688e+01, 2.04529474e-01, ...,
         3.15037666e+01, 1.62701326e+01, 2.01938024e+01],
        [0.00000000e+00, 0.00000000e+00, 9.75471029e-18, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 1.01570512e-15, 3.96759812e-18, ...,
         5.07852559e-16, 2.53926279e-16, 2.53926279e-16],
        ...,
        [0.00000000e+00, 2.15894788e-16, 8.43339014e-19, ...,
         1.07947394e-16, 5.39736969e-17, 5.39736969e-17],
        [0.00000000e+00, 0.00000000e+00, 1.02526137e-18, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]],

       [[9.64827570e-01, 4.70984811e+01, 2.04529474e-01, ...,
         3.15037666e+01, 1.62693947e+01, 2.02488017e+01],
        [7.40520390e-03, 1.92148767e+00, 9.75471029e-18, ...,
         0.00000000e+00, 7.37854077e-04, 5.49992849e-02],
        [7.40520390e-03, 

In [18]:
print(x_train_rolled.shape)
print(x_train_fft.shape)

(417720, 30, 10)
(417720, 16, 10)


In [16]:
num_vars = x_train_rolled.shape[2]
for i in range(num_vars):
    mean_s = np.mean(x_train_rolled[:,:,i])
    sd_s = np.std(x_train_rolled[:,:,i])
    x_train_rolled[:,:,i] = (x_train_rolled[:,:,i]-mean_s)/sd_s

num_vars_fft = x_train_fft.shape[2]
for i in range(num_vars_fft):
    mean_s = np.mean(x_train_fft[:,:,i])
    sd_s = np.std(x_train_fft[:,:,i])
    x_train_fft[:,:,i] = (x_train_fft[:,:,i]-mean_s)/sd_s

In [19]:
x_train = x_train_rolled.transpose(0,2,1)
x_train_fft = x_train_fft.transpose(0,2,1)

In [20]:
print(x_train.shape)
print(x_train_fft.shape)

(417720, 10, 30)
(417720, 10, 16)


In [21]:
def build_datasets(data, target, train_size, valid_pct = 0.2, seed = None):
    x, x_fft = data
    idx = np.arange(train_size)
    train_idx, val_idx = train_test_split(idx, test_size = valid_pct, random_state = seed)
    train_ds = TensorDataset(torch.tensor(x[:train_size][train_idx]).float(),
                            torch.tensor(x_fft[:train_size][train_idx]).float(),
                            torch.tensor(target[:train_size][train_idx]).long())
    val_ds = TensorDataset(torch.tensor(x[:train_size][val_idx]).float(),
                            torch.tensor(x_fft[:train_size][val_idx]).float(),
                            torch.tensor(target[:train_size][val_idx]).long())
#     test_ds = TensorDataset(torch.tensor(x[train_size:]).float(),
#                             torch.tensor(x_fft[train_size:]).float(),
#                             torch.tensor(target[train_size:]).long())
    return train_ds, val_ds #, test_ds

In [22]:
def build_loaders(data, batch_size = 128, jobs = 8):
#     train_ds, valid_ds, test_ds = data
    train_ds, valid_ds = data
    train_dl = DataLoader(train_ds, batch_size = batch_size, shuffle = False, num_workers = jobs)
    valid_dl = DataLoader(valid_ds, batch_size = batch_size, shuffle = False, num_workers = jobs)
#     test_dl = DataLoader(test_ds, batch_size = batch_size, shuffle = True, num_workers = jobs)
    return train_dl, valid_dl #, test_dl

In [23]:
class _SepConv1d(nn.Module):
    """A simple separable convolution implementation.
    
    The separable convlution is a method to reduce number of the parameters 
    in the deep learning network for slight decrease in predictions quality.
    """
    def __init__(self, ni, no, kernel, stride, pad):
        super().__init__()
        self.depthwise = nn.Conv1d(ni, ni, kernel, stride, padding=pad, groups=ni)
        self.pointwise = nn.Conv1d(ni, no, kernel_size=1)

    def forward(self, x):
        return self.pointwise(self.depthwise(x))

In [24]:
class SepConv1d(nn.Module):
    """Implementes a 1-d convolution with 'batteries included'.
    
    The module adds (optionally) activation function and dropout layers right after
    a separable convolution layer.
    """
    def __init__(self, ni, no, kernel, stride, pad, drop=None,
                 activ=lambda: nn.ReLU(inplace=True)):
    
        super().__init__()
        assert drop is None or (0.0 < drop < 1.0)
        layers = [_SepConv1d(ni, no, kernel, stride, pad)]
        if activ:
            layers.append(activ())
        if drop is not None:
            layers.append(nn.Dropout(drop))
        self.layers = nn.Sequential(*layers)
        
    def forward(self, x): 
        return self.layers(x)

In [25]:
class Flatten(nn.Module):
    """Converts N-dimensional tensor into 'flat' one."""

    def __init__(self, keep_batch_dim=True):
        super().__init__()
        self.keep_batch_dim = keep_batch_dim

    def forward(self, x):
        if self.keep_batch_dim:
            return x.view(x.size(0), -1)
        return x.view(-1)

In [26]:
class PrintSize(nn.Module):
    def __init__(self):
        super(PrintSize, self).__init__()
        
    def forward(self, x):
        print(x.shape)
        return x

In [58]:
class Classifier(nn.Module):
    def __init__(self, raw_ni, fft_ni, no, drop=.5):
        super().__init__()
        #PKS [[4,8,2],[3,8,2],[5,8,2],[2,8,2]]
        self.raw = nn.Sequential( #kernel, stride, pad
            SepConv1d(raw_ni,  32, 8, 2, 4, drop=drop),
            SepConv1d(    32,  64, 8, 2, 3, drop=drop),
            SepConv1d(    64, 128, 8, 2, 5, drop=drop),
            SepConv1d(   128, 256, 8, 2, 2, drop=drop),
            Flatten(),
#             PrintSize(),
            nn.Dropout(drop), nn.Linear(512, 256), nn.ReLU(inplace=True),
            nn.Dropout(drop), nn.Linear( 256, 64), nn.ReLU(inplace=True))
        #PKS [3,8,2],[5,8,2],[4,8,2],[5,8,2],[3,8,2]
        self.fft = nn.Sequential(
            SepConv1d(fft_ni,  32, 8, 2, 3, drop=drop),
            SepConv1d(    32,  64, 8, 2, 5, drop=drop),
            SepConv1d(    64, 128, 8, 2, 4, drop=drop),
            SepConv1d(   128, 128, 8, 2, 5, drop=drop),
            SepConv1d(   128, 256, 8, 2, 3, drop=drop),
            Flatten(),
#             PrintSize(),
            nn.Dropout(drop), nn.Linear(512, 256), nn.ReLU(inplace=True),
            nn.Dropout(drop), nn.Linear( 256, 64), nn.ReLU(inplace=True))
        
        self.out = nn.Sequential(
            nn.Linear(128, 64), nn.ReLU(inplace=True), nn.Linear(64, no))
        
    def forward(self, t_raw, t_fft):
        raw_out = self.raw(t_raw)
        fft_out = self.fft(t_fft)
        t_in = torch.cat([raw_out, fft_out], dim=1)
        out = self.out(t_in)
        return out

In [32]:
seed = 1
train_size = len(x_train)
datasets = build_datasets((x_train, x_train_fft), y_train_transformed,
                          train_size = 400000, valid_pct = 0.2, seed=seed)

In [59]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')

In [60]:
raw_feat = x_train.shape[1]
fft_feat = x_train_fft.shape[1]
# trn_dl, val_dl, tst_dl = build_loaders(datasets, batch_size=256)
trn_dl, val_dl = build_loaders(datasets, batch_size=1024)

lr = 0.0002
n_epochs = 3000
iterations_per_epoch = len(trn_dl)
num_classes = 20
best_acc = 0
patience, trials = 3000, 0
base = 1
step = 2
loss_history = []
acc_history = []
gstep = -1
model = Classifier(raw_feat, fft_feat, num_classes,drop = .5).to(device)
criterion = nn.CrossEntropyLoss(reduction='sum')
opt = torch.optim.Adam(model.parameters(), lr=lr, weight_decay= 0.0001)
lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(opt, 0.004, steps_per_epoch = len(datasets[0]),epochs = n_epochs,
                                                  pct_start = 0.4,base_momentum = 0.85, max_momentum=0.95,
                                                  div_factor = 10.0, last_epoch = gstep)
print('Start model training')

for epoch in range(1, n_epochs + 1):
    
    model.train()
    epoch_loss = 0
    for i, batch in enumerate(trn_dl):
        
        x_raw, x_fft, y_batch = [t.to(device) for t in batch]
        opt.zero_grad()
        out = model(x_raw, x_fft)
        loss = criterion(out, y_batch)
        epoch_loss += loss.item()
        loss.backward()
        opt.step()
        lr_scheduler.step()
        
    epoch_loss /= train_size
    loss_history.append(epoch_loss)
    
    model.eval()
    correct, total = 0, 0
    for batch in val_dl:
        x_raw, x_fft, y_batch = [t.to(device) for t in batch]
        out = model(x_raw, x_fft)
        preds = F.log_softmax(out, dim=1).argmax(dim=1)
        total += y_batch.size(0)
        correct += (preds == y_batch).sum().item()
    
    acc = correct / total
    acc_history.append(acc)

    if epoch % base == 0:
        print(f'Epoch: {epoch:3d}. Loss: {epoch_loss:.4f}. Acc.: {acc:2.2%}')
        base *= step

    if acc > best_acc:
        trials = 0
        best_acc = acc
        torch.save(model.state_dict(), 'best.pth')
        print(f'Epoch {epoch} best model saved with accuracy: {best_acc:2.2%}')
    else:
        trials += 1
        if trials >= patience:
            print(f'Early stopping on epoch {epoch}')
            break
            
print('Done!')

Start model training
Epoch:   1. Loss: 0.7588. Acc.: 78.95%
Epoch 1 best model saved with accuracy: 78.95%
Epoch:   2. Loss: 0.3360. Acc.: 92.35%
Epoch 2 best model saved with accuracy: 92.35%
Epoch 3 best model saved with accuracy: 92.67%
Epoch:   4. Loss: 0.2571. Acc.: 93.09%
Epoch 4 best model saved with accuracy: 93.09%
Epoch 5 best model saved with accuracy: 93.17%
Epoch 6 best model saved with accuracy: 93.25%
Epoch:   8. Loss: 0.2339. Acc.: 93.34%
Epoch 8 best model saved with accuracy: 93.34%
Epoch 9 best model saved with accuracy: 93.35%
Epoch 10 best model saved with accuracy: 93.40%
Epoch 11 best model saved with accuracy: 93.51%
Epoch 12 best model saved with accuracy: 93.52%
Epoch 13 best model saved with accuracy: 93.59%
Epoch 14 best model saved with accuracy: 93.62%
Epoch:  16. Loss: 0.2173. Acc.: 93.68%
Epoch 16 best model saved with accuracy: 93.68%
Epoch 18 best model saved with accuracy: 93.71%
Epoch 19 best model saved with accuracy: 93.73%
Epoch 20 best model save

KeyboardInterrupt: 

In [ ]:
test = np.load('./np_x_train_rolled_new.npy', allow_pickle= True)

In [ ]:
np.array(test,dtype = np.float64)

In [ ]:
x_train_rolled.to_numpy(dtype=np.float64).dtype

In [ ]:
np.vstack(x_train_rolled[:, 0:1]).astype(np.float)


In [ ]:
x_train_rolled.drop('id', axis=1, inplace=True)
x_train_rolled

In [ ]:
y_train.shape

In [ ]:
# x_train_rolled = pd.concat([roll_time_series(temp, column_id="dummy_id", column_sort="id",
#             max_timeshift = 29, min_timeshift = 29),x_train_rolled]).reset_index(drop = True)

In [ ]:
# np.save("np_x_train_rolled_new.npy",np.array(x_train_rolled))

In [ ]:
print(x_train)

In [ ]:
# x_train.drop(x_train.index[0:29])
x_train.head(100)

In [ ]:
# 

## no need now - additional features calculators 

In [ ]:
x_features = extract_features(x_train_rolled.loc[0:299999],column_id='id',column_sort='ids', 
                 default_fc_parameters=EfficientFCParameters())

In [ ]:
x_features.shape

In [ ]:
len(x_train_rolled.loc[0:299999,'id'].value_counts())

In [ ]:
from tsfresh.feature_selection.significance_tests import target_real_feature_real_test
x_features_dropped = x_features.dropna(axis=1, how='any')
keeps = {}
for col in x_features_dropped.columns:
    p_value = target_real_feature_real_test(x_features_dropped[col].reset_index().drop(columns=["level_0","level_1"],
                                                                       axis = 1).squeeze(), 
                                     pd.Series(y_train_transformed[29:(29+60130)]))
    if p_value < 0.01:
        keeps[col] = p_value

In [ ]:
best_features = sorted(list(dict(sorted(keeps.items(), key=lambda item: item[1])).keys())[:255])
x_features = x_features[best_features]

In [ ]:
x_features['ids'] = x_features.index
x_train_rolled_temp = x_train_rolled.loc[0:299999]

In [ ]:
x_train_temp = x_train_rolled_temp.merge(x_features, how='left', left_on="id", right_on="ids")

In [ ]:
x_features

In [ ]:
x_minimal = extract_features(x_train_rolled.loc[0:299999],column_id='id',column_sort='ids', 
                             default_fc_parameters=MinimalFCParameters())
x_minimal.shape

In [ ]:
len(y_train_transformed)

In [ ]:
tsfresh.feature_selection.significance_tests.target_real_feature_real_test(x_minimal['load__standard_deviation'].reset_index().drop(columns=["level_0","level_1"], axis = 1).squeeze(), 
                                                                               pd.Series(y_train_transformed[29:(29+60130)]))

In [ ]:
x_minimal.head(10000)

In [ ]:
pd.set_option('display.max_rows', 10000)
data.head(10000)

In [ ]:
pd.reset_option('all')

In [ ]:
data.columns

In [ ]:
def absfft(x):
    return np.abs(np.fft.rfft(x))
x_train_fft = np.copy(x_train)
x_train_fft = np.apply_along_axis(absfft, 1, x_train_fft)
x_train_fft

In [ ]:
x_train_fft.shape

In [ ]:
x_train.shape

In [ ]:
num_vars = x_train.shape[2]
for i in range(num_vars):
    mean_s = np.mean(x_train[:,:,i])
    sd_s = np.std(x_train[:,:,i])
    x_train[:,:,i] = (x_train[:,:,i]-mean_s)/sd_s

num_vars_fft = x_train_fft.shape[2]
for i in range(num_vars_fft):
    mean_s = np.mean(x_train_fft[:,:,i])
    sd_s = np.std(x_train_fft[:,:,i])
    x_train_fft[:,:,i] = (x_train_fft[:,:,i]-mean_s)/sd_s

In [ ]:

x_train = x_train.transpose(0,2,1)
x_train_fft = x_train_fft.transpose(0,2,1)

In [ ]:
x_train.shape

In [ ]:
x_train_fft.shape

In [ ]:
def build_datasets(data, target, train_size, valid_pct = 0.2, seed = None):
    x, x_fft = data
    idx = np.arange(train_size)
    train_idx, val_idx = train_test_split(idx, test_size = valid_pct, random_state = seed)
    train_ds = TensorDataset(torch.tensor(x[:train_size][train_idx]).float(),
                            torch.tensor(x_fft[:train_size][train_idx]).float(),
                            torch.tensor(target[:train_size][train_idx]).long())
    val_ds = TensorDataset(torch.tensor(x[:train_size][val_idx]).float(),
                            torch.tensor(x_fft[:train_size][val_idx]).float(),
                            torch.tensor(target[:train_size][val_idx]).long())
#     test_ds = TensorDataset(torch.tensor(x[train_size:]).float(),
#                             torch.tensor(x_fft[train_size:]).float(),
#                             torch.tensor(target[train_size:]).long())
    return train_ds, val_ds #, test_ds

In [ ]:
def build_loaders(data, batch_size = 128, jobs = 8):
#     train_ds, valid_ds, test_ds = data
    train_ds, valid_ds = data
    train_dl = DataLoader(train_ds, batch_size = batch_size, shuffle = False, num_workers = jobs)
    valid_dl = DataLoader(valid_ds, batch_size = batch_size, shuffle = False, num_workers = jobs)
#     test_dl = DataLoader(test_ds, batch_size = batch_size, shuffle = True, num_workers = jobs)
    return train_dl, valid_dl #, test_dl

In [ ]:
class _SepConv1d(nn.Module):
    """A simple separable convolution implementation.
    
    The separable convlution is a method to reduce number of the parameters 
    in the deep learning network for slight decrease in predictions quality.
    """
    def __init__(self, ni, no, kernel, stride, pad):
        super().__init__()
        self.depthwise = nn.Conv1d(ni, ni, kernel, stride, padding=pad, groups=ni)
        self.pointwise = nn.Conv1d(ni, no, kernel_size=1)

    def forward(self, x):
        return self.pointwise(self.depthwise(x))

In [ ]:
class SepConv1d(nn.Module):
    """Implementes a 1-d convolution with 'batteries included'.
    
    The module adds (optionally) activation function and dropout layers right after
    a separable convolution layer.
    """
    def __init__(self, ni, no, kernel, stride, pad, drop=None,
                 activ=lambda: nn.ReLU(inplace=True)):
    
        super().__init__()
        assert drop is None or (0.0 < drop < 1.0)
        layers = [_SepConv1d(ni, no, kernel, stride, pad)]
        if activ:
            layers.append(activ())
        if drop is not None:
            layers.append(nn.Dropout(drop))
        self.layers = nn.Sequential(*layers)
        
    def forward(self, x): 
        return self.layers(x)

In [ ]:
class Flatten(nn.Module):
    """Converts N-dimensional tensor into 'flat' one."""

    def __init__(self, keep_batch_dim=True):
        super().__init__()
        self.keep_batch_dim = keep_batch_dim

    def forward(self, x):
        if self.keep_batch_dim:
            return x.view(x.size(0), -1)
        return x.view(-1)

In [ ]:
class PrintSize(nn.Module):
    def __init__(self):
        super(PrintSize, self).__init__()
        
    def forward(self, x):
        print(x.shape)
        return x

In [ ]:
class Classifier(nn.Module):
    def __init__(self, raw_ni, fft_ni, no, drop=.5):
        super().__init__()
        #PKS [[3,8,2],[3,8,4],[3,8,2],[3,8,2]]
        self.raw = nn.Sequential( #kernel, stride, pad
            SepConv1d(raw_ni,  32, 8, 2, 3, drop=drop),
            SepConv1d(    32,  64, 8, 4, 3, drop=drop),
            SepConv1d(    64, 128, 8, 2, 3, drop=drop),
            SepConv1d(   128, 256, 8, 2, 3),
            Flatten(),
#             PrintSize(),
            nn.Dropout(drop), nn.Linear(512, 256), nn.ReLU(inplace=True),
            nn.Dropout(drop), nn.Linear( 256, 64), nn.ReLU(inplace=True))
        #PKS [[5,8,3],[4,8,2],[5,8,3],[5,8,3],[4,8,3]]
        self.fft = nn.Sequential(
            SepConv1d(fft_ni,  32, 8, 3, 5, drop=drop),
            SepConv1d(    32,  64, 8, 2, 4, drop=drop),
            SepConv1d(    64, 128, 8, 3, 5, drop=drop),
            SepConv1d(   128, 128, 8, 3, 5, drop=drop),
            SepConv1d(   128, 256, 8, 3, 4),
            Flatten(),
#             PrintSize(),
            nn.Dropout(drop), nn.Linear(512, 256), nn.ReLU(inplace=True),
            nn.Dropout(drop), nn.Linear( 256, 64), nn.ReLU(inplace=True))
        
        self.out = nn.Sequential(
            nn.Linear(128, 64), nn.ReLU(inplace=True), nn.Linear(64, no))
        
    def forward(self, t_raw, t_fft):
        raw_out = self.raw(t_raw)
        fft_out = self.fft(t_fft)
        t_in = torch.cat([raw_out, fft_out], dim=1)
        out = self.out(t_in)
        return out

In [ ]:
seed = 1
train_size = len(x_train)
datasets = build_datasets((x_train, x_train_fft), y_train_transformed, train_size,valid_pct = 0.2, seed=seed)

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')

In [ ]:
raw_feat = x_train.shape[1]
fft_feat = x_train_fft.shape[1]
# trn_dl, val_dl, tst_dl = build_loaders(datasets, batch_size=256)
trn_dl, val_dl = build_loaders(datasets, batch_size=1024)

lr = 0.0002
n_epochs = 3000
iterations_per_epoch = len(trn_dl)
num_classes = 20
best_acc = 0
patience, trials = 3000, 0
base = 1
step = 2
loss_history = []
acc_history = []
gstep = -1
model = Classifier(raw_feat, fft_feat, num_classes,drop = .5).to(device)
criterion = nn.CrossEntropyLoss(reduction='sum')
opt = torch.optim.Adam(model.parameters(), lr=lr, weight_decay= 0.0001)
lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(opt, 0.004, steps_per_epoch = len(datasets[0]),epochs = n_epochs,
                                                  pct_start = 0.4,base_momentum = 0.85, max_momentum=0.95,
                                                  div_factor = 10.0, last_epoch = gstep)
print('Start model training')

for epoch in range(1, n_epochs + 1):
    
    model.train()
    epoch_loss = 0
    for i, batch in enumerate(trn_dl):
        
        x_raw, x_fft, y_batch = [t.to(device) for t in batch]
        opt.zero_grad()
        out = model(x_raw, x_fft)
        loss = criterion(out, y_batch)
        epoch_loss += loss.item()
        loss.backward()
        opt.step()
        lr_scheduler.step()
        
    epoch_loss /= train_size
    loss_history.append(epoch_loss)
    
    model.eval()
    correct, total = 0, 0
    for batch in val_dl:
        x_raw, x_fft, y_batch = [t.to(device) for t in batch]
        out = model(x_raw, x_fft)
        preds = F.log_softmax(out, dim=1).argmax(dim=1)
        total += y_batch.size(0)
        correct += (preds == y_batch).sum().item()
    
    acc = correct / total
    acc_history.append(acc)

    if epoch % base == 0:
        print(f'Epoch: {epoch:3d}. Loss: {epoch_loss:.4f}. Acc.: {acc:2.2%}')
        base *= step

    if acc > best_acc:
        trials = 0
        best_acc = acc
        torch.save(model.state_dict(), 'best.pth')
        print(f'Epoch {epoch} best model saved with accuracy: {best_acc:2.2%}')
    else:
        trials += 1
        if trials >= patience:
            print(f'Early stopping on epoch {epoch}')
            break
            
print('Done!')

In [ ]:
y_batch

In [ ]:
y_train.encoded.unique()

In [ ]:
criterion(out, torch.tensor([1]))

In [ ]:
l = 9
pks = [[3,8,2],[2,8,4],[2,8,4],[2,8,4]]
for i, layer in enumerate(pks):
    p,k,s = layer
    l = (l + 2*p - k)/s + 1
    print(f"length: {l}")

In [ ]:
l = 6
pks = [[4,8,2],[4,8,2],[4,8,4],[4,8,4],[3,8,2]]
for i, layer in enumerate(pks):
    p,k,s = layer
    l = (l + 2*p - k)/s + 1
    print(f"length: {l}")

In [61]:
l = 30
pks = [[4,8,2],[3,8,2],[5,8,2],[2,8,2]]
for i, layer in enumerate(pks):
    p,k,s = layer
#     w = (w + 2*p - k)/s + 1
#     h = (h + 2*p - k)/s + 1
    l = (l + 2*p - k)/s + 1
    print(f"length: {l}")

length: 16.0
length: 8.0
length: 6.0
length: 2.0


In [62]:
l = 12
pks = [[3,8,2],[5,8,2],[4,8,2],[5,8,2],[3,8,2]]
for i, layer in enumerate(pks):
    p,k,s = layer
#     w = (w + 2*p - k)/s + 1
#     h = (h + 2*p - k)/s + 1
    l = (l + 2*p - k)/s + 1
    print(f"length: {l}")

length: 6.0
length: 5.0
length: 3.5
length: 3.75
length: 1.875


In [ ]:
        #PKS [[4,8,2],[3,8,2],[5,8,2],[2,8,2]]
        self.raw = nn.Sequential( #kernel, stride, pad
            SepConv1d(raw_ni,  32, 8, 2, 4, drop=drop),
            SepConv1d(    32,  64, 8, 2, 3, drop=drop),
            SepConv1d(    64, 128, 8, 2, 5, drop=drop),
            SepConv1d(   128, 256, 8, 2, 2, drop=drop),
            Flatten(),
#             PrintSize(),
            nn.Dropout(drop), nn.Linear(512, 256), nn.ReLU(inplace=True),
            nn.Dropout(drop), nn.Linear( 256, 64), nn.ReLU(inplace=True))
        #PKS [3,8,2],[5,8,2],[4,8,2],[5,8,2],[3,8,2]

In [ ]:
count_hourly = x_train.loc[:,:].groupby(["dayofweek",'hourofday']).count()
count_hourly.reset_index(inplace=True)

In [ ]:
count_hourly

In [ ]:
x_train.sort_values(by=["dayofweek",'hourofday']).head(1000)

In [ ]:
x_train = x_train.sort_values(by=["dayofweek",'hourofday'])

In [ ]:
x_train[(x_train.dayofweek == "Fri") & (x_train.hourofday == 11)].first_valid_index()

In [ ]:
min_hourly_obs = min(count_hourly['var'].values) 
drop_idx = []
for _, row in count_hourly.iterrows():
    drop_idx = [*drop_idx,*x_train[(x_train.dayofweek == row[0]) & (x_train.hourofday == row[1])].index.values[0:row[2] - min_hourly_obs]]
drop_idx

In [ ]:
x_train.drop(index = drop_idx, inplace = True)
x_train.loc[:,:].groupby(["dayofweek",'hourofday']).count()

In [ ]:
x_train

In [ ]:
a = [123]
b = list(range(1,4))
a + b

In [ ]:
min_hourly_obs

In [ ]:
df = pd.DataFrame({
   "id": [1, 1, 1, 1, 2, 2],
   "time": [1, 2, 3, 4, 8, 9],
   "x": [1, 2, 3, 4, 10, 11],
   "y": [5, 6, 7, 8, 12, 13],
})

In [ ]:
x_train.shape

In [ ]:
from tsfresh.utilities.dataframe_functions import roll_time_series
df_rolled = roll_time_series(x_train[['load','dummy_id','id']], column_id="dummy_id", column_sort="id",
                            max_timeshift = 29, min_timeshift = 29)
df_rolled

In [ ]:
from tsfresh.utilities.dataframe_functions import roll_time_series
df_rolled = roll_time_series(df, column_id="id", column_sort="time", max_timeshift=2, min_timeshift=2)
df_rolled

In [ ]:
from tsfresh import extract_features
df_features = extract_features(df_rolled, column_id="id", column_sort="time", )


In [ ]:
df_features

In [ ]:
x_train.loc[0:500, ['load','dummy_id','id']]